In [89]:
from __future__ import division

import csv
import numpy as np
from random import randrange
import math
import copy
from collections import Counter

In [90]:
data = []
with open('irisData.csv', 'rb') as csv_file:
    iris_data_reader = csv.reader(csv_file, delimiter=',', quotechar='|')
    for row in iris_data_reader:
        if row[4] == 'Iris-setosa':
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 1])
        elif row[4] == 'Iris-versicolor':
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 2])
        else:
            data.append([float(1), float(row[0]), float(row[1]), float(row[2]), float(row[3]), 3])

In [90]:
# def normalize_dataset(data):
#     minmax = list()
#     for i in range(len(data[0])):
#         col_values = [row[i] for row in data]
#         value_min = min(col_values)
#         value_max = max(col_values)
#         minmax.append([value_min, value_max])
# 
#     for row in data:
#         for i in range(len(row) - 1):
#             if minmax[i][1] != minmax[i][0]:
#                 row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
# 
# 
# normalize_dataset(data)
np.random.shuffle(data)

In [92]:
def cross_validation_split(dataset, n_folds):
    """ Split data set for n folds. Need for k-fold validation"""
    dataset_split = []
    dataset_copy = copy.copy(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [93]:
def euclidean_distance(elem1, elem2):
    distance = 0
    for index in range(len(elem1)-1):
        distance += pow((elem1[index] - elem2[index]), 2)
    return math.sqrt(distance)

In [94]:
def k_nnn(train_data, test_data, k):
    score = 0
    for test_elem in test_data:
        distances = []
        for train_elem in train_data:
            distance = euclidean_distance(train_elem, test_elem)
            distances.append((distance, train_elem[-1]))
        sorted_by_distance = sorted(distances, key=lambda tup: tup[0])

        cnt = Counter()
        for distance in sorted_by_distance[0:k]:
            cnt[distance[1]] += 1

        if test_elem[-1] == cnt.most_common(1)[0][0]:
            score += 1
    return score / len(test_data)

In [96]:
best_score = None
best_k = 0
for k in xrange(1, 30):
    folds = cross_validation_split(data, 5)
    scores = []
    for test_set in folds:
        train_set = [item for item in data if item not in test_set]
        score = k_nnn(train_set, test_set, k)
    
        scores.append(score)

    if sum(scores)/len(scores) > best_score:
        best_score = sum(scores)/len(scores)
        best_k = k

print best_score
print best_k

0.986666666667
12


В алгоритме кросс валидации у меня сделано так, что выборка разбивается на фолды каждый раз рандомно (для разных k в k-nn), поэтому при различных запусках кросс валидации получается разный результат точности.  Но в целом лучшая точность получется при k > 7 (точность около 98%).

Если выполнять предварительную нормализацию данных, то результат точности получается хуже (примерно на 2%). Вот это я не знаю почему..


Проклятие размерности — проблема, связанная с экспоненциальным возрастанием количества данных из-за увеличения размерности пространства. Напимер, как было показано на лекции по k-nn. Т.е. чем больше размерность, тем гораздо больше размер обучающей выборки нам нужен.
В многомерных пространствах расстояние между объекта не так как мы привыкли, c ростом числа признаков необходимый объем обучающей выборки растет экспоненциально, и это становится серьезной проблемой для метрических алгоритмов. Объекты часто оказываются на примерно одинаковом расстоянии друг от друга. Проклятие размерности может существенно мешать применению метрических алгоритмов.

Способы борьбы:
- понижение размерности пространства, спроецировать данные на подпространство меньшей размерности (как сделано в svm)
- отбирать признаки на которых будет алгоритм убачаться, тем самым уменьшая размерность пространства
